In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup

In [21]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

url = 'https://www.google.com/finance/?hl=ja'
driver.get(url)

time.sleep(1)


In [24]:
stock_code = 'VIX'

stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys(stock_code)
stock_code_input.send_keys(Keys.RETURN)

time.sleep(3)

In [23]:
driver.find_element(By.XPATH, r'//*[@id="6monthTab"]').click()

logs = driver.get_log('performance')

In [25]:
current_url = driver.current_url
new_url = f"{current_url}&window=6M"
driver.get(new_url)
logs = driver.get_log('performance')

In [ ]:
print(logs)

In [26]:
batch_request_urls = []
headers = {}
post_data = None


for entry in logs:
    log = json.loads(entry['message'])['message']
    try:
        if 'Network.requestWillBeSent' in log['method']:
            request_url = log['params']['request']['url']
            if f'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=AiCwsd&source-path=%2Ffinance%2Fquote%2F{stock_code}' in request_url:
                batch_request_urls.append(request_url)
                headers = log['params']['request']['headers']
                if 'postData' in log['params']['request']:
                    post_data = log['params']['request']['postData']
    except KeyError:
        continue

print(batch_request_urls)

['https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=AiCwsd&source-path=%2Ffinance%2Fquote%2FVIX%3AINDEXCBOE&f.sid=919186302077798105&bl=boq_finance-ui_20240723.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=74341&rt=c', 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=AiCwsd&source-path=%2Ffinance%2Fquote%2FVIX%3AINDEXCBOE&f.sid=919186302077798105&bl=boq_finance-ui_20240723.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=174341&rt=c']


In [20]:
target_string = fr'[["wrb.fr","AiCwsd","[[[[\"{stock_code}\",'

response = requests.post(batch_request_urls[1], headers=headers, data=post_data)

print(response.text)
if response.text in target_string:
    print(True)

)]}'

117962
[["wrb.fr","AiCwsd","[[[[\".INX\",\"INDEXSP\"],\"/m/016yss\",null,[[[1],[[[2024,2,20,16,null,null,null,[-14400]],[4975.51,0,0,2,2,2]],[[2024,2,21,16,null,null,null,[-14400]],[4981.8,6.289999999999964,0.0012641920124771056,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[5087.03,111.51999999999953,0.02241378270770223,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[5088.8,113.28999999999996,0.022769525134106846,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[5069.53,94.01999999999953,0.01889655532799643,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[5078.18,102.67000000000007,0.020635070575679693,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[5069.76,94.25,0.018942781744986946,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[5096.27,120.76000000000022,0.024270878764187032,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[5137.08,161.5699999999997,0.03247305301366085,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[5130.95,155.4399999999996,0.03124101850865531,2,2,3]],[[20

In [10]:
# RequestsでGETリクエストを送信し、特定の文字列が含まれるレスポンスを探す
target_string = fr'[["wrb.fr","AiCwsd","[[[[\"{stock_code}\",'

for url in batch_request_urls:
    if post_data:
        response = requests.post(url, headers=headers, data=post_data)
    else:
        response = requests.get(url, headers=headers)

    print(f"Checking URL: {url}")

    print(response.text)
    if target_string in response.text and stock_code in response.text:
        print("Found target string in the response!")
        print("Status Code:", response.status_code)
        print("Response Text:", response.text)
        # try:
        #     data = response.json()
        #     print(data)
        # except json.JSONDecodeError:
        #     print("レスポンスのJSONデコードに失敗しました")
        # break
    else:
        print("対象の文字列を含むレスポンスが見つかりませんでした")

Checking URL: https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=AiCwsd&source-path=%2Ffinance%2Fquote%2FVIX%3AINDEXCBOE&f.sid=6247315020053961812&bl=boq_finance-ui_20240723.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=60905&rt=c
)]}'

117957
[["wrb.fr","AiCwsd","[[[[\".INX\",\"INDEXSP\"],\"/m/016yss\",null,[[[1],[[[2024,2,20,16,null,null,null,[-14400]],[4975.51,0,0,2,2,2]],[[2024,2,21,16,null,null,null,[-14400]],[4981.8,6.289999999999964,0.0012641920124771056,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[5087.03,111.51999999999953,0.02241378270770223,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[5088.8,113.28999999999996,0.022769525134106846,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[5069.53,94.01999999999953,0.01889655532799643,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[5078.18,102.67000000000007,0.020635070575679693,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[5069.76,94.25,0.018942781744986946,2,2,3]],[[2024,2,29,16,null,nu

In [2]:
NASDAQ = r''')
]
}'

12245
[["wrb.fr","AiCwsd","[[[[\"NDX\",\"INDEXNASDAQ\"],\"/m/02fhwv\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[17596.27,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[17476.71,-119.56000000000131,-0.006794621814736948,2,2,4]],[[2024,1,31,16,null,null,null,[-14400]],[17137.24,-459.02999999999884,-0.026086778618423043,2,2,3]],[[2024,2,1,16,null,null,null,[-14400]],[17344.71,-251.5600000000013,-0.014296211640307935,2,2,3]],[[2024,2,2,16,null,null,null,[-14400]],[17642.73,46.45999999999913,0.0026403322976971326,2,2,4]],[[2024,2,5,16,null,null,null,[-14400]],[17613.04,16.770000000000437,9.53042889203248E-4,2,2,5]],[[2024,2,6,16,null,null,null,[-14400]],[17572.73,-23.540000000000873,-0.0013377835188935424,2,2,4]],[[2024,2,7,16,null,null,null,[-14400]],[17755.07,158.79999999999927,0.009024639881065662,2,2,4]],[[2024,2,8,16,null,null,null,[-14400]],[17783.17,186.89999999999782,0.010621569230296979,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[17962.41,366.1399999999994,0.02080781892980725,2,2,3]],[[2024,2,12,16,null,null,null,[-14400]],[17882.66,286.3899999999994,0.016275608410191444,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[17600.42,4.149999999997817,2.3584543769775168E-4,2,2,5]],[[2024,2,14,16,null,null,null,[-14400]],[17807.63,211.36000000000058,0.01201163655706582,2,2,3]],[[2024,2,15,16,null,null,null,[-14400]],[17845.72,249.45000000000073,0.014176299863550668,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[17685.98,89.70999999999913,0.005098239570090657,2,2,4]],[[2024,2,20,16,null,null,null,[-14400]],[17546.1,-50.17000000000189,-0.0028511724359765954,2,2,4]],[[2024,2,21,16,null,null,null,[-14400]],[17478.91,-117.36000000000058,-0.006669595317644056,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[18004.7,408.4300000000003,0.02321116918528758,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[17937.61,341.34000000000015,0.019398429326215164,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[17933.33,337.0600000000013,0.019155195959143688,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[17971.05,374.77999999999884,0.021298832082026407,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[17874.5,278.22999999999956,0.015811873766428883,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[18043.85,447.5799999999981,0.02543607253128067,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[18302.91,706.6399999999994,0.04015851086622332,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[18226.48,630.2099999999991,0.035814976696765796,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[17897.87,301.59999999999854,0.017139996146910596,2,2,3]],[[2024,3,6,16,null,null,null,[-14400]],[18017.58,421.3100000000013,0.02394314249554032,2,2,3]],[[2024,3,7,16,null,null,null,[-14400]],[18297.99,701.7200000000012,0.039878906154543044,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[18018.45,422.1800000000003,0.02399258479211789,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[17951.69,355.41999999999825,0.0201985989076093,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[18219.12,622.8499999999985,0.03539670623376423,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[18068.47,472.2000000000007,0.02683523269420171,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[18014.81,418.5400000000009,0.02378572276965521,2,2,3]],[[2024,3,15,16,null,null,null,[-14400]],[17808.25,211.97999999999956,0.012046871297155565,2,2,3]],[[2024,3,18,16,null,null,null,[-14400]],[17985.01,388.73999999999796,0.022092182036306442,2,2,3]],[[2024,3,19,16,null,null,null,[-14400]],[18032.21,435.9399999999987,0.024774568701207624,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[18240.11,643.8400000000001,0.03658957267648201,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[18320.38,724.1100000000006,0.04115133491359251,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[18339.44,743.1699999999983,0.04223451902022407,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[18277.06,680.7900000000009,0.03868944952538242,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[18210.54,614.2700000000004,0.034909102895102226,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[18280.84,684.5699999999997,0.038904267779478245,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[18254.69,658.4199999999983,0.03741815737085179,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[18293.2,696.9300000000003,0.03960668937223629,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[18121.78,525.5099999999984,0.029864852039665133,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[18160.19,563.9199999999983,0.03204770101845438,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[17878.78,282.5099999999984,0.01605510713350036,2,2,3]],[[2024,4,5,16,null,null,null,[-14400]],[18108.46,512.1899999999987,0.029107873429993894,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[18100.19,503.91999999999825,0.028637887461376656,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[18169.91,573.6399999999994,0.032600090814701034,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[18011.66,415.3899999999994,0.023606707557908547,2,2,3]],[[2024,4,11,16,null,null,null,[-14400]],[18307.98,711.7099999999991,0.04044664011179637,2,2,3]],[[2024,4,12,16,null,null,null,[-14400]],[18003.49,407.22000000000116,0.02314240461188656,2,2,3]],[[2024,4,15,16,null,null,null,[-14400]],[17706.83,110.56000000000131,0.006283149781175289,2,2,4]],[[2024,4,16,16,null,null,null,[-14400]],[17713.66,117.38999999999942,0.006671300224422529,2,2,4]],[[2024,4,17,16,null,null,null,[-14400]],[17493.62,-102.65000000000146,-0.005833622693900551,2,2,4]],[[2024,4,18,16,null,null,null,[-14400]],[17394.31,-201.95999999999913,-0.01147743243312356,2,2,3]],[[2024,4,19,16,null,null,null,[-14400]],[17037.65,-558.619999999999,-0.03174650082091256,2,2,3]],[[2024,4,22,16,null,null,null,[-14400]],[17210.89,-385.380000000001,-0.021901232477110264,2,2,3]],[[2024,4,23,16,null,null,null,[-14400]],[17471.47,-124.79999999999927,-0.007092412198721619,2,2,4]],[[2024,4,24,16,null,null,null,[-14400]],[17526.8,-69.47000000000116,-0.003947995796836554,2,2,4]],[[2024,4,25,16,null,null,null,[-14400]],[17430.5,-165.77000000000044,-0.009420746555946257,2,2,4]],[[2024,4,26,16,null,null,null,[-14400]],[17718.3,122.02999999999884,0.006934992472836506,2,2,4]],[[2024,4,29,16,null,null,null,[-14400]],[17782.72,186.45000000000073,0.010595995628619062,2,2,3]],[[2024,4,30,16,null,null,null,[-14400]],[17440.69,-155.58000000000175,-0.00884164655350263,2,2,4]],[[2024,5,1,16,null,null,null,[-14400]],[17318.55,-277.72000000000116,-0.015782890351193814,2,2,3]],[[2024,5,2,16,null,null,null,[-14400]],[17541.54,-54.72999999999956,-0.0031103182663143702,2,2,4]],[[2024,5,3,16,null,null,null,[-14400]],[17890.79,294.52000000000044,0.016737638147175535,2,2,3]],[[2024,5,6,16,null,null,null,[-14400]],[18093.57,497.2999999999993,0.028261671365579142,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[18091.45,495.1800000000003,0.028141191286562454,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[18085.01,488.73999999999796,0.027775204631435978,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[18113.46,517.1899999999987,0.02939202455975037,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[18161.18,564.9099999999999,0.03210396294214625,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[18198.61,602.3400000000001,0.03423111829950325,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[18322.77,726.5,0.041287159153616075,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[18596.65,1000.380000000001,0.05685182143715691,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[18557.96,961.6899999999987,0.05465305999510116,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[18546.23,949.9599999999991,0.05398644144469249,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[18674.19,1077.9199999999983,0.061258437157420194,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[18713.8,1117.5299999999988,0.06350948240735103,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[18705.2,1108.9300000000003,0.06302074246416998,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[18623.39,1027.119999999999,0.05837146167909443,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[18808.35,1212.079999999998,0.06888278027104597,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[18869.44,1273.1699999999983,0.07235453877441061,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[18736.76,1140.489999999998,0.06481430439519273,2,2,3]],[[2024,5,30,16,null,null,null,[-14400]],[18538.66,942.3899999999994,0.0535562366342412,2,2,3]],[[2024,5,31,16,null,null,null,[-14400]],[18536.65,940.380000000001,0.05344200788007919,2,2,3]],[[2024,6,3,16,null,null,null,[-14400]],[18600.97,1004.7000000000007,0.05709732801326649,2,2,3]],[[2024,6,4,16,null,null,null,[-14400]],[18654.84,1058.5699999999997,0.06015877228526271,2,2,3]],[[2024,6,5,16,null,null,null,[-14400]],[19035.05,1438.7799999999988,0.08176619249420466,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[19021.19,1424.9199999999983,0.08097852556251968,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[19000.95,1404.6800000000003,0.07982828178926558,2,2,3]],[[2024,6,10,16,null,null,null,[-14400]],[19074.68,1478.4099999999999,0.08401837434865456,2,2,3]],[[2024,6,11,16,null,null,null,[-14400]],[19210.19,1613.9199999999983,0.0917194382673145,2,2,3]],[[2024,6,12,16,null,null,null,[-14400]],[19465.18,1868.9099999999999,0.1062105775826354,2,2,2]],[[2024,6,13,16,null,null,null,[-14400]],[19576.92,1980.6499999999978,0.11256078703043303,2,2,2]],[[2024,6,14,16,null,null,null,[-14400]],[19659.8,2063.529999999999,0.11727087615727644,2,2,2]],[[2024,6,17,16,null,null,null,[-14400]],[19902.75,2306.4799999999996,0.1310777795521437,2,2,2]],[[2024,6,18,16,null,null,null,[-14400]],[19908.86,2312.59,0.13142501223270614,2,2,2]],[[2024,6,20,16,null,null,null,[-14400]],[19752.3,2156.029999999999,0.12252767205777126,2,2,2]],[[2024,6,21,16,null,null,null,[-14400]],[19700.43,2104.16,0.11957988823767764,2,2,2]],[[2024,6,24,16,null,null,null,[-14400]],[19474.62,1878.3499999999985,0.10674705491561555,2,2,2]],[[2024,6,25,16,null,null,null,[-14400]],[19701.13,2104.8600000000006,0.11961966939584358,2,2,2]],[[2024,6,26,16,null,null,null,[-14400]],[19751.05,2154.779999999999,0.12245663427533214,2,2,2]],[[2024,6,27,16,null,null,null,[-14400]],[19789.03,2192.7599999999984,0.12461504625696232,2,2,2]],[[2024,6,28,16,null,null,null,[-14400]],[19682.87,2086.5999999999985,0.11858194946997282,2,2,2]],[[2024,7,1,16,null,null,null,[-14400]],[19812.22,2215.9500000000007,0.12593293919677298,2,2,2]],[[2024,7,2,16,null,null,null,[-14400]],[20011.89,2415.619999999999,0.13728023041246804,2,2,2]],[[2024,7,3,13,5,null,null,[-14400]],[20186.63,2590.3600000000006,0.14721074409519747,2,2,2]],[[2024,7,5,16,null,null,null,[-14400]],[20391.97,2795.7000000000007,0.15888026269203648,2,2,2]],[[2024,7,8,16,null,null,null,[-14400]],[20439.54,2843.2700000000004,0.16158367654053957,2,2,2]],[[2024,7,9,16,null,null,null,[-14400]],[20453.02,2856.75,0.162349747986363,2,2,2]],[[2024,7,10,16,null,null,null,[-14400]],[20675.38,3079.1100000000006,0.1749865170288931,2,2,2]],[[2024,7,11,16,null,null,null,[-14400]],[20211.36,2615.09,0.14861615558297298,2,2,2]],[[2024,7,12,16,null,null,null,[-14400]],[20331.49,2735.220000000001,0.15544317062650215,2,2,2]],[[2024,7,15,16,null,null,null,[-14400]],[20386.88,2790.6100000000006,0.15859099684194436,2,2,2]],[[2024,7,16,16,null,null,null,[-14400]],[20398.62,2802.3499999999985,0.15925818369461248,2,2,2]],[[2024,7,17,16,null,null,null,[-14400]],[19799.14,2202.869999999999,0.12518959984132993,2,2,2]],[[2024,7,18,16,null,null,null,[-14400]],[19705.09,2108.8199999999997,0.11984471709061066,2,2,2]],[[2024,7,19,16,null,null,null,[-14400]],[19522.62,1926.3499999999985,0.10947490576127773,2,2,2]],[[2024,7,22,16,null,null,null,[-14400]],[19822.87,2226.5999999999985,0.12653818110315415,2,2,2]],[[2024,7,23,16,null,null,null,[-14400]],[19754.34,2158.0699999999997,0.12264360571871195,2,2,2]],[[2024,7,24,16,null,null,null,[-14400]],[19032.39,1436.119999999999,0.08161502409317423,2,2,3]],[[2024,7,25,16,null,null,null,[-14400]],[18830.59,1234.3199999999997,0.07014668449620287,2,2,3]],[[2024,7,26,16,null,null,null,[-14400]],[19023.66,1427.3899999999994,0.08111889622061945,2,2,3]],[[2024,7,26,17,15,59,null,[-14400]],[19023.656,1427.3859999999986,0.0811186688997156,2,2,3]]]]],null,-14400,18830.584,\"ナスダック100指数\",86400,1]]]",null,null,null,"generic"]]
55
[["di",78],["af.httprm",78,"6461234555725867655",32]]
27
[["e",4,null,null,12356]]
'''

In [3]:
SP500 = r''')
]
}'

12046
[["wrb.fr","AiCwsd","[[[[\".INX\",\"INDEXSP\"],\"/m/016yss\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[4927.93,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[4924.97,-2.9600000000000364,-6.006578827215558E-4,2,2,5]],[[2024,1,31,16,null,null,null,[-14400]],[4845.65,-82.28000000000065,-0.016696665739976146,2,2,3]],[[2024,2,1,16,null,null,null,[-14400]],[4906.19,-21.74000000000069,-0.004411588638637458,2,2,4]],[[2024,2,2,16,null,null,null,[-14400]],[4958.61,30.67999999999938,0.006225737784424572,2,2,4]],[[2024,2,5,16,null,null,null,[-14400]],[4942.81,14.88000000000011,0.003019523410438076,2,2,4]],[[2024,2,6,16,null,null,null,[-14400]],[4954.23,26.299999999999272,0.005336926457964961,2,2,4]],[[2024,2,7,16,null,null,null,[-14400]],[4995.06,67.13000000000011,0.013622352590235679,2,2,3]],[[2024,2,8,16,null,null,null,[-14400]],[4997.91,69.97999999999956,0.014200688727315437,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[5026.61,98.67999999999938,0.020024635090189872,2,2,3]],[[2024,2,12,16,null,null,null,[-14400]],[5021.84,93.90999999999985,0.01905668302918261,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[4953.17,25.23999999999978,0.005121825999963429,2,2,4]],[[2024,2,14,16,null,null,null,[-14400]],[5000.62,72.6899999999996,0.014750615369942268,2,2,3]],[[2024,2,15,16,null,null,null,[-14400]],[5029.73,101.79999999999927,0.02065776096657202,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[5005.57,77.63999999999942,0.01575509392381779,2,2,3]],[[2024,2,20,16,null,null,null,[-14400]],[4975.51,47.57999999999993,0.009655169614828117,2,2,4]],[[2024,2,21,16,null,null,null,[-14400]],[4981.8,53.86999999999989,0.0109315676156114,2,2,3]],[[2024,2,22,16,null,null,null,[-14400]],[5087.03,159.09999999999945,0.03228536119628311,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[5088.8,160.8699999999999,0.03264453837615386,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[5069.53,141.59999999999945,0.02873417438965234,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[5078.18,150.25,0.030489475296929948,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[5069.76,141.82999999999993,0.02878084713053958,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[5096.27,168.34000000000015,0.034160387830184304,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[5137.08,209.14999999999964,0.04244175546324717,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[5130.95,203.01999999999953,0.04119782545612448,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[5078.65,150.71999999999935,0.030584850028307898,2,2,3]],[[2024,3,6,16,null,null,null,[-14400]],[5104.76,176.82999999999993,0.03588322074380113,2,2,3]],[[2024,3,7,16,null,null,null,[-14400]],[5157.36,229.42999999999938,0.04655707365973124,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[5123.69,195.7599999999993,0.03972459024377361,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[5117.94,190.0099999999993,0.03855777172159493,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[5175.27,247.34000000000015,0.05019145970011752,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[5165.31,237.3800000000001,0.048170327094743656,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[5150.48,222.54999999999927,0.045160949932324376,2,2,3]],[[2024,3,15,16,null,null,null,[-14400]],[5117.09,189.15999999999985,0.038385285505272974,2,2,3]],[[2024,3,18,16,null,null,null,[-14400]],[5149.42,221.48999999999978,0.04494584947432284,2,2,3]],[[2024,3,19,16,null,null,null,[-14400]],[5178.51,250.57999999999993,0.050848936571745114,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[5224.62,296.6899999999996,0.060205806494816196,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[5241.53,313.59999999999945,0.0636372675748234,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[5234.18,306.25,0.06214576911603858,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[5218.19,290.2599999999993,0.058900998999579804,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[5203.58,275.64999999999964,0.05593626532844412,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[5248.49,320.5599999999995,0.06504962529906055,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[5254.35,326.4200000000001,0.06623876556688103,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[5243.77,315.84000000000015,0.06409181948607227,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[5205.81,277.8800000000001,0.056388787990089166,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[5211.49,283.5599999999995,0.057541401765041196,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[5147.21,219.27999999999975,0.04449738531188546,2,2,3]],[[2024,4,5,16,null,null,null,[-14400]],[5204.34,276.40999999999985,0.056090488298332124,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[5202.39,274.46000000000004,0.055694784625593304,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[5209.91,281.97999999999956,0.05722078032764255,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[5160.64,232.71000000000004,0.04722266752977417,2,2,3]],[[2024,4,11,16,null,null,null,[-14400]],[5199.06,271.1300000000001,0.055019044507531575,2,2,3]],[[2024,4,12,16,null,null,null,[-14400]],[5123.41,195.47999999999956,0.03966777125486757,2,2,3]],[[2024,4,15,16,null,null,null,[-14400]],[5061.82,133.88999999999942,0.027169622945131,2,2,3]],[[2024,4,16,16,null,null,null,[-14400]],[5051.41,123.47999999999956,0.025057174107586667,2,2,3]],[[2024,4,17,16,null,null,null,[-14400]],[5022.21,94.27999999999975,0.01913176526452278,2,2,3]],[[2024,4,18,16,null,null,null,[-14400]],[5011.12,83.1899999999996,0.016881327453920732,2,2,3]],[[2024,4,19,16,null,null,null,[-14400]],[4967.23,39.29999999999927,0.00797495094289068,2,2,4]],[[2024,4,22,16,null,null,null,[-14400]],[5010.6,82.67000000000007,0.0167758064745238,2,2,3]],[[2024,4,23,16,null,null,null,[-14400]],[5070.55,142.6199999999999,0.028941157849238907,2,2,3]],[[2024,4,24,16,null,null,null,[-14400]],[5071.63,143.69999999999982,0.029160316806448105,2,2,3]],[[2024,4,25,16,null,null,null,[-14400]],[5048.42,120.48999999999978,0.024450428476053793,2,2,3]],[[2024,4,26,16,null,null,null,[-14400]],[5099.96,172.02999999999975,0.03490918093398237,2,2,3]],[[2024,4,29,16,null,null,null,[-14400]],[5116.17,188.23999999999978,0.03819859454172437,2,2,3]],[[2024,4,30,16,null,null,null,[-14400]],[5035.69,107.75999999999931,0.021867193730430282,2,2,3]],[[2024,5,1,16,null,null,null,[-14400]],[5018.39,90.46000000000004,0.018356591915875433,2,2,3]],[[2024,5,2,16,null,null,null,[-14400]],[5064.2,136.26999999999953,0.027652584350832808,2,2,3]],[[2024,5,3,16,null,null,null,[-14400]],[5127.79,199.85999999999967,0.04055658258132718,2,2,3]],[[2024,5,6,16,null,null,null,[-14400]],[5180.74,252.8099999999995,0.05130145923338998,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[5187.7,259.7699999999995,0.05271381695762714,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[5187.67,259.7399999999998,0.05270772920881583,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[5214.08,286.14999999999964,0.05806697741242258,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[5222.68,294.75,0.05981213207168121,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[5221.42,293.4899999999998,0.05955644662160375,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[5246.68,318.75,0.06468233112077484,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[5308.15,380.21999999999935,0.07715612843526579,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[5297.1,369.1700000000001,0.07491380762307907,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[5303.27,375.34000000000015,0.07616585462861691,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[5308.13,380.1999999999998,0.0771520699360583,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[5321.41,393.47999999999956,0.07984691340989006,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[5307.01,379.0799999999999,0.07692479398043396,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[5267.84,339.90999999999985,0.06897622328239238,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[5304.72,376.78999999999996,0.07646009582116628,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[5306.04,378.1099999999997,0.07672795676886637,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[5266.95,339.0199999999995,0.06879562006765508,2,2,3]],[[2024,5,30,16,null,null,null,[-14400]],[5235.48,307.5499999999993,0.062409571564531,2,2,3]],[[2024,5,31,16,null,null,null,[-14400]],[5277.51,349.5799999999999,0.07093850764925637,2,2,3]],[[2024,6,3,16,null,null,null,[-14400]],[5283.4,355.46999999999935,0.07213373566588797,2,2,3]],[[2024,6,4,16,null,null,null,[-14400]],[5291.34,363.40999999999985,0.07374495985129656,2,2,3]],[[2024,6,5,16,null,null,null,[-14400]],[5354.03,426.09999999999945,0.08646632561744981,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[5352.96,425.02999999999975,0.08624919590984444,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[5346.99,419.0599999999995,0.08503773389638235,2,2,3]],[[2024,6,10,16,null,null,null,[-14400]],[5360.79,432.8599999999997,0.08783809834961123,2,2,3]],[[2024,6,11,16,null,null,null,[-14400]],[5375.32,447.3899999999994,0.09078659802391661,2,2,3]],[[2024,6,12,16,null,null,null,[-14400]],[5421.03,493.09999999999945,0.10006229796283621,2,2,2]],[[2024,6,13,16,null,null,null,[-14400]],[5433.74,505.8099999999995,0.10264147420925206,2,2,2]],[[2024,6,14,16,null,null,null,[-14400]],[5431.6,503.6700000000001,0.10220721479404132,2,2,2]],[[2024,6,17,16,null,null,null,[-14400]],[5473.23,545.2999999999993,0.11065498089461483,2,2,2]],[[2024,6,18,16,null,null,null,[-14400]],[5487.03,559.0999999999995,0.1134553453478437,2,2,2]],[[2024,6,20,16,null,null,null,[-14400]],[5473.17,545.2399999999998,0.1106428053969922,2,2,2]],[[2024,6,21,16,null,null,null,[-14400]],[5464.62,536.6899999999996,0.10890779698575255,2,2,2]],[[2024,6,24,16,null,null,null,[-14400]],[5447.87,519.9399999999996,0.10550880389940595,2,2,2]],[[2024,6,25,16,null,null,null,[-14400]],[5469.3,541.3699999999999,0.10985748580032587,2,2,2]],[[2024,6,26,16,null,null,null,[-14400]],[5477.9,549.9699999999993,0.1116026404595843,2,2,2]],[[2024,6,27,16,null,null,null,[-14400]],[5482.87,554.9399999999996,0.1126111775126675,2,2,2]],[[2024,6,28,16,null,null,null,[-14400]],[5460.48,532.5499999999993,0.10806768764978383,2,2,2]],[[2024,7,1,16,null,null,null,[-14400]],[5475.09,547.1599999999999,0.1110324213209197,2,2,2]],[[2024,7,2,16,null,null,null,[-14400]],[5509.01,581.0799999999999,0.11791563597697205,2,2,2]],[[2024,7,3,13,null,null,null,[-14400]],[5537.02,609.0900000000001,0.12359956411718513,2,2,2]],[[2024,7,5,16,null,null,null,[-14400]],[5567.19,639.2599999999993,0.12972181017181642,2,2,2]],[[2024,7,8,16,null,null,null,[-14400]],[5572.85,644.9200000000001,0.13087036544756114,2,2,2]],[[2024,7,9,16,null,null,null,[-14400]],[5576.98,649.0499999999993,0.13170844553392586,2,2,2]],[[2024,7,10,16,null,null,null,[-14400]],[5633.91,705.9799999999996,0.1432609635282968,2,2,2]],[[2024,7,11,16,null,null,null,[-14400]],[5584.54,656.6099999999997,0.13324255823439043,2,2,2]],[[2024,7,12,16,null,null,null,[-14400]],[5615.35,687.4200000000001,0.13949467626366446,2,2,2]],[[2024,7,15,16,null,null,null,[-14400]],[5631.22,703.29,0.14271509538487762,2,2,2]],[[2024,7,16,16,null,null,null,[-14400]],[5667.2,739.2699999999995,0.1500163354593104,2,2,2]],[[2024,7,17,16,null,null,null,[-14400]],[5588.27,660.3400000000001,0.13399946833660384,2,2,2]],[[2024,7,18,16,null,null,null,[-14400]],[5544.59,616.6599999999999,0.12513570606725335,2,2,2]],[[2024,7,19,16,null,null,null,[-14400]],[5505,577.0699999999997,0.11710190688585262,2,2,2]],[[2024,7,22,16,null,null,null,[-14400]],[5564.41,636.4799999999996,0.12915767878196313,2,2,2]],[[2024,7,23,16,null,null,null,[-14400]],[5555.74,627.8099999999995,0.12739831937547802,2,2,2]],[[2024,7,24,16,null,null,null,[-14400]],[5427.13,499.1999999999998,0.10130014022114758,2,2,2]],[[2024,7,25,16,null,null,null,[-14400]],[5399.22,471.28999999999996,0.09563650457697247,2,2,3]],[[2024,7,26,16,null,null,null,[-14400]],[5459.1,531.1700000000001,0.10778765120446111,2,2,2]],[[2024,7,26,17,9,24,null,[-14400]],[5459.1,531.1700000000001,0.10778765120446111,2,2,2]]]]],null,-14400,5399.22,\"S\\u0026P 500\",86400,1]]]",null,null,null,"generic"]]
57
[["di",69],["af.httprm",68,"-4510631086066940026",304]]
27
[["e",4,null,null,12145]]
'''

In [4]:
VIX = r''')
]
}'

11502
[["wrb.fr","AiCwsd","[[[[\"VIX\",\"INDEXCBOE\"],\"/m/09fld6\",null,[[[1],[[[2024,1,29,15,16,null,null,[-18000]],[13.6,0,0,2,2,2]],[[2024,1,30,15,16,null,null,[-18000]],[13.31,-0.28999999999999915,-0.021323529411764644,2,2,3]],[[2024,1,31,15,16,null,null,[-18000]],[14.35,0.75,0.05514705882352941,2,2,3]],[[2024,2,1,15,16,null,null,[-18000]],[13.88,0.28000000000000114,0.02058823529411773,2,2,3]],[[2024,2,2,15,16,null,null,[-18000]],[13.85,0.25,0.01838235294117647,2,2,3]],[[2024,2,5,15,16,null,null,[-18000]],[13.67,0.07000000000000028,0.0051470588235294325,2,3,4]],[[2024,2,6,15,16,null,null,[-18000]],[13.06,-0.5399999999999991,-0.03970588235294111,2,2,3]],[[2024,2,7,15,16,null,null,[-18000]],[12.83,-0.7699999999999996,-0.0566176470588235,2,2,3]],[[2024,2,8,15,16,null,null,[-18000]],[12.79,-0.8100000000000005,-0.0595588235294118,2,2,3]],[[2024,2,9,15,16,null,null,[-18000]],[12.93,-0.6699999999999999,-0.04926470588235294,2,2,3]],[[2024,2,12,15,16,null,null,[-18000]],[13.93,0.33000000000000007,0.02426470588235295,2,2,3]],[[2024,2,13,15,16,null,null,[-18000]],[15.85,2.25,0.16544117647058823,2,2,2]],[[2024,2,14,15,16,null,null,[-18000]],[14.38,0.7800000000000011,0.057352941176470676,2,2,3]],[[2024,2,15,15,16,null,null,[-18000]],[14.01,0.41000000000000014,0.030147058823529423,2,2,3]],[[2024,2,16,15,16,null,null,[-18000]],[14.24,0.6400000000000006,0.047058823529411806,2,2,3]],[[2024,2,20,15,16,null,null,[-18000]],[15.42,1.8200000000000003,0.13382352941176473,2,2,2]],[[2024,2,21,15,16,null,null,[-18000]],[15.34,1.7400000000000002,0.12794117647058825,2,2,2]],[[2024,2,22,15,16,null,null,[-18000]],[14.54,0.9399999999999995,0.06911764705882349,2,2,3]],[[2024,2,23,15,16,null,null,[-18000]],[13.75,0.15000000000000036,0.011029411764705909,2,2,3]],[[2024,2,26,15,16,null,null,[-18000]],[13.74,0.14000000000000057,0.010294117647058865,2,2,3]],[[2024,2,27,15,16,null,null,[-18000]],[13.43,-0.16999999999999993,-0.012499999999999995,2,2,3]],[[2024,2,28,15,16,null,null,[-18000]],[13.84,0.2400000000000002,0.01764705882352943,2,2,3]],[[2024,2,29,15,16,null,null,[-18000]],[13.4,-0.1999999999999993,-0.014705882352941124,2,2,3]],[[2024,3,1,15,16,null,null,[-18000]],[13.11,-0.4900000000000002,-0.0360294117647059,2,2,3]],[[2024,3,4,15,16,null,null,[-18000]],[13.49,-0.10999999999999943,-0.008088235294117606,2,2,4]],[[2024,3,5,15,16,null,null,[-18000]],[14.46,0.8600000000000012,0.06323529411764715,2,2,3]],[[2024,3,6,15,16,null,null,[-18000]],[14.5,0.9000000000000004,0.06617647058823532,2,2,3]],[[2024,3,7,15,16,null,null,[-18000]],[14.44,0.8399999999999999,0.06176470588235293,2,2,3]],[[2024,3,8,15,16,null,null,[-18000]],[14.74,1.1400000000000006,0.08382352941176476,2,2,3]],[[2024,3,11,15,16,null,null,[-18000]],[15.22,1.620000000000001,0.1191176470588236,2,2,2]],[[2024,3,12,15,16,null,null,[-18000]],[13.84,0.2400000000000002,0.01764705882352943,2,2,3]],[[2024,3,13,15,16,null,null,[-18000]],[13.75,0.15000000000000036,0.011029411764705909,2,2,3]],[[2024,3,14,15,16,null,null,[-18000]],[14.4,0.8000000000000007,0.05882352941176476,2,2,3]],[[2024,3,15,15,16,null,null,[-18000]],[14.41,0.8100000000000005,0.0595588235294118,2,2,3]],[[2024,3,18,15,16,null,null,[-18000]],[14.33,0.7300000000000004,0.053676470588235325,2,2,3]],[[2024,3,19,15,16,null,null,[-18000]],[13.82,0.22000000000000064,0.01617647058823534,2,2,3]],[[2024,3,20,15,16,null,null,[-18000]],[13.04,-0.5600000000000005,-0.041176470588235335,2,2,3]],[[2024,3,21,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,3,22,15,16,null,null,[-18000]],[13.06,-0.5399999999999991,-0.03970588235294111,2,2,3]],[[2024,3,25,15,16,null,null,[-18000]],[13.19,-0.41000000000000014,-0.030147058823529423,2,2,3]],[[2024,3,26,15,16,null,null,[-18000]],[13.24,-0.35999999999999943,-0.026470588235294076,2,2,3]],[[2024,3,27,15,16,null,null,[-18000]],[12.78,-0.8200000000000003,-0.060294117647058845,2,2,3]],[[2024,3,28,15,16,null,null,[-18000]],[13.01,-0.5899999999999999,-0.04338235294117646,2,2,3]],[[2024,4,1,15,16,null,null,[-18000]],[13.65,0.05000000000000071,0.0036764705882353465,2,3,4]],[[2024,4,2,15,16,null,null,[-18000]],[14.61,1.0099999999999998,0.07426470588235293,2,2,3]],[[2024,4,3,15,16,null,null,[-18000]],[14.33,0.7300000000000004,0.053676470588235325,2,2,3]],[[2024,4,4,15,16,null,null,[-18000]],[16.35,2.7500000000000018,0.20220588235294132,2,2,2]],[[2024,4,5,15,16,null,null,[-18000]],[16.03,2.4300000000000015,0.1786764705882354,2,2,2]],[[2024,4,8,15,16,null,null,[-18000]],[15.19,1.5899999999999999,0.11691176470588234,2,2,2]],[[2024,4,9,15,16,null,null,[-18000]],[14.98,1.3800000000000008,0.10147058823529417,2,2,2]],[[2024,4,10,15,16,null,null,[-18000]],[15.8,2.200000000000001,0.16176470588235303,2,2,2]],[[2024,4,11,15,16,null,null,[-18000]],[14.91,1.3100000000000005,0.09632352941176474,2,2,3]],[[2024,4,12,15,16,null,null,[-18000]],[17.31,3.709999999999999,0.27279411764705874,2,2,2]],[[2024,4,15,15,16,null,null,[-18000]],[19.23,5.630000000000001,0.4139705882352942,2,2,2]],[[2024,4,16,15,16,null,null,[-18000]],[18.4,4.799999999999999,0.35294117647058815,2,2,2]],[[2024,4,17,15,16,null,null,[-18000]],[18.21,4.610000000000001,0.3389705882352942,2,2,2]],[[2024,4,18,15,16,null,null,[-18000]],[18,4.4,0.3235294117647059,2,2,2]],[[2024,4,19,15,16,null,null,[-18000]],[18.71,5.110000000000001,0.37573529411764717,2,2,2]],[[2024,4,22,15,16,null,null,[-18000]],[16.94,3.3400000000000016,0.24558823529411777,2,2,2]],[[2024,4,23,15,16,null,null,[-18000]],[15.69,2.09,0.15367647058823528,2,2,2]],[[2024,4,24,15,16,null,null,[-18000]],[15.97,2.370000000000001,0.17426470588235302,2,2,2]],[[2024,4,25,15,16,null,null,[-18000]],[15.37,1.7699999999999996,0.1301470588235294,2,2,2]],[[2024,4,26,15,16,null,null,[-18000]],[15.03,1.4299999999999997,0.1051470588235294,2,2,2]],[[2024,4,29,15,16,null,null,[-18000]],[14.67,1.0700000000000003,0.07867647058823532,2,2,3]],[[2024,4,30,15,16,null,null,[-18000]],[15.65,2.0500000000000007,0.1507352941176471,2,2,2]],[[2024,5,1,15,16,null,null,[-18000]],[15.39,1.790000000000001,0.1316176470588236,2,2,2]],[[2024,5,2,15,16,null,null,[-18000]],[14.68,1.08,0.07941176470588236,2,2,3]],[[2024,5,3,15,16,null,null,[-18000]],[13.49,-0.10999999999999943,-0.008088235294117606,2,2,4]],[[2024,5,6,15,16,null,null,[-18000]],[13.49,-0.10999999999999943,-0.008088235294117606,2,2,4]],[[2024,5,7,15,16,null,null,[-18000]],[13.23,-0.3699999999999992,-0.027205882352941118,2,2,3]],[[2024,5,8,15,16,null,null,[-18000]],[13,-0.5999999999999996,-0.044117647058823505,2,2,3]],[[2024,5,9,15,16,null,null,[-18000]],[12.69,-0.9100000000000001,-0.06691176470588237,2,2,3]],[[2024,5,10,15,16,null,null,[-18000]],[12.55,-1.049999999999999,-0.0772058823529411,2,2,3]],[[2024,5,13,15,16,null,null,[-18000]],[13.6,0,0,2,2,2]],[[2024,5,14,15,16,null,null,[-18000]],[13.42,-0.17999999999999972,-0.013235294117647038,2,2,3]],[[2024,5,15,15,16,null,null,[-18000]],[12.45,-1.1500000000000004,-0.0845588235294118,2,2,3]],[[2024,5,16,15,16,null,null,[-18000]],[12.42,-1.1799999999999997,-0.08676470588235292,2,2,3]],[[2024,5,17,15,16,null,null,[-18000]],[11.99,-1.6099999999999994,-0.11838235294117644,2,2,2]],[[2024,5,20,15,16,null,null,[-18000]],[12.15,-1.4499999999999993,-0.10661764705882348,2,2,2]],[[2024,5,21,15,16,null,null,[-18000]],[11.86,-1.7400000000000002,-0.12794117647058825,2,2,2]],[[2024,5,22,15,16,null,null,[-18000]],[12.29,-1.3100000000000005,-0.09632352941176474,2,2,3]],[[2024,5,23,15,16,null,null,[-18000]],[12.77,-0.8300000000000001,-0.06102941176470589,2,2,3]],[[2024,5,24,15,16,null,null,[-18000]],[11.93,-1.67,-0.12279411764705882,2,2,2]],[[2024,5,28,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,5,29,15,16,null,null,[-18000]],[14.28,0.6799999999999997,0.04999999999999998,2,2,3]],[[2024,5,30,15,16,null,null,[-18000]],[14.47,0.870000000000001,0.0639705882352942,2,2,3]],[[2024,5,31,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,6,3,15,16,null,null,[-18000]],[13.11,-0.4900000000000002,-0.0360294117647059,2,2,3]],[[2024,6,4,15,16,null,null,[-18000]],[13.16,-0.4399999999999995,-0.03235294117647055,2,2,3]],[[2024,6,5,15,16,null,null,[-18000]],[12.63,-0.9699999999999989,-0.07132352941176462,2,2,3]],[[2024,6,6,15,16,null,null,[-18000]],[12.58,-1.0199999999999996,-0.07499999999999997,2,2,3]],[[2024,6,7,15,16,null,null,[-18000]],[12.22,-1.379999999999999,-0.10147058823529405,2,2,2]],[[2024,6,10,15,16,null,null,[-18000]],[12.74,-0.8599999999999994,-0.06323529411764701,2,2,3]],[[2024,6,11,15,16,null,null,[-18000]],[12.85,-0.75,-0.05514705882352941,2,2,3]],[[2024,6,12,15,16,null,null,[-18000]],[12.04,-1.5600000000000005,-0.11470588235294121,2,2,2]],[[2024,6,13,15,16,null,null,[-18000]],[11.94,-1.6600000000000001,-0.12205882352941178,2,2,2]],[[2024,6,14,15,16,null,null,[-18000]],[12.66,-0.9399999999999995,-0.06911764705882349,2,2,3]],[[2024,6,17,15,16,null,null,[-18000]],[12.75,-0.8499999999999996,-0.06249999999999997,2,2,3]],[[2024,6,18,15,16,null,null,[-18000]],[12.3,-1.299999999999999,-0.09558823529411757,2,2,3]],[[2024,6,20,15,16,null,null,[-18000]],[13.28,-0.3200000000000003,-0.023529411764705903,2,2,3]],[[2024,6,21,15,16,null,null,[-18000]],[13.2,-0.40000000000000036,-0.02941176470588238,2,2,3]],[[2024,6,24,15,16,null,null,[-18000]],[13.33,-0.2699999999999996,-0.019852941176470556,2,2,3]],[[2024,6,25,15,16,null,null,[-18000]],[12.84,-0.7599999999999998,-0.05588235294117646,2,2,3]],[[2024,6,26,15,16,null,null,[-18000]],[12.55,-1.049999999999999,-0.0772058823529411,2,2,3]],[[2024,6,27,15,16,null,null,[-18000]],[12.24,-1.3599999999999994,-0.09999999999999996,2,2,3]],[[2024,6,28,15,16,null,null,[-18000]],[12.44,-1.1600000000000001,-0.08529411764705884,2,2,3]],[[2024,7,1,15,16,null,null,[-18000]],[12.22,-1.379999999999999,-0.10147058823529405,2,2,2]],[[2024,7,2,15,16,null,null,[-18000]],[12.03,-1.5700000000000003,-0.11544117647058826,2,2,2]],[[2024,7,3,13,null,null,null,[-18000]],[12.09,-1.5099999999999998,-0.11102941176470588,2,2,2]],[[2024,7,5,15,16,null,null,[-18000]],[12.48,-1.1199999999999992,-0.08235294117647053,2,2,3]],[[2024,7,8,15,16,null,null,[-18000]],[12.37,-1.2300000000000004,-0.09044117647058827,2,2,3]],[[2024,7,9,15,16,null,null,[-18000]],[12.51,-1.0899999999999999,-0.0801470588235294,2,2,3]],[[2024,7,10,15,16,null,null,[-18000]],[12.85,-0.75,-0.05514705882352941,2,2,3]],[[2024,7,11,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,7,12,15,16,null,null,[-18000]],[12.46,-1.1399999999999988,-0.08382352941176462,2,2,3]],[[2024,7,15,15,16,null,null,[-18000]],[13.12,-0.4800000000000004,-0.03529411764705886,2,2,3]],[[2024,7,16,15,16,null,null,[-18000]],[13.19,-0.41000000000000014,-0.030147058823529423,2,2,3]],[[2024,7,17,15,16,null,null,[-18000]],[14.48,0.8800000000000008,0.06470588235294124,2,2,3]],[[2024,7,18,15,16,null,null,[-18000]],[15.93,2.33,0.1713235294117647,2,2,2]],[[2024,7,19,15,16,null,null,[-18000]],[16.52,2.92,0.21470588235294116,2,2,2]],[[2024,7,22,15,16,null,null,[-18000]],[14.91,1.3100000000000005,0.09632352941176474,2,2,3]],[[2024,7,23,15,16,null,null,[-18000]],[14.72,1.120000000000001,0.08235294117647067,2,2,3]],[[2024,7,24,15,16,null,null,[-18000]],[18.04,4.4399999999999995,0.32647058823529407,2,2,2]],[[2024,7,25,15,16,null,null,[-18000]],[18.46,4.860000000000001,0.3573529411764707,2,2,2]],[[2024,7,26,15,16,null,null,[-18000]],[16.39,2.790000000000001,0.2051470588235295,2,2,2]]]]],null,-18000,18.46,\"VIX指数\",86400,1]]]",null,null,null,"generic"]]
55
[["di",64],["af.httprm",63,"7889371759034056131",25]]
27
[["e",4,null,null,11603]]
'''

In [5]:
DJI = r''')
]
}'

12313
[["wrb.fr","AiCwsd","[[[[\".DJI\",\"INDEXDJX\"],\"/m/0cqyw\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[38333.45,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[38467.31,133.86000000000058,0.003491989372206274,2,2,4]],[[2024,1,31,16,null,null,null,[-14400]],[38150.3,-183.14999999999418,-0.004777811545790796,2,2,4]],[[2024,2,1,16,null,null,null,[-14400]],[38519.84,186.38999999999942,0.0048623330276820744,2,2,4]],[[2024,2,2,16,null,null,null,[-14400]],[38654.42,320.97000000000116,0.008373104951419744,2,2,4]],[[2024,2,5,16,null,null,null,[-14400]],[38380.12,46.67000000000553,0.0012174745555123666,2,2,4]],[[2024,2,6,16,null,null,null,[-14400]],[38521.36,187.9100000000035,0.004901985080915063,2,2,4]],[[2024,2,7,16,null,null,null,[-14400]],[38677.36,343.9100000000035,0.008971537912710792,2,2,4]],[[2024,2,8,16,null,null,null,[-14400]],[38726.33,392.88000000000466,0.010249012285614905,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[38671.69,338.24000000000524,0.008823625319401339,2,2,4]],[[2024,2,12,16,null,null,null,[-14400]],[38797.38,463.9300000000003,0.01210248490548073,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[38272.75,-60.69999999999709,-0.0015834734416024932,2,2,4]],[[2024,2,14,16,null,null,null,[-14400]],[38424.27,90.81999999999971,0.00236921018066466,2,2,4]],[[2024,2,15,16,null,null,null,[-14400]],[38773.12,439.67000000000553,0.011469617266382378,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[38627.99,294.5400000000009,0.007683628788955883,2,2,4]],[[2024,2,20,16,null,null,null,[-14400]],[38563.8,230.35000000000582,0.006009112146180577,2,2,4]],[[2024,2,21,16,null,null,null,[-14400]],[38612.24,278.7900000000009,0.0072727604742072755,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[39069.11,735.6600000000035,0.019191072027172185,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[39131.53,798.0800000000017,0.02081941489743297,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[39069.23,735.7800000000061,0.019194202452427478,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[38972.41,638.9600000000064,0.016668471009001446,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[38949.02,615.5699999999997,0.01605829895300318,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[38996.39,662.9400000000023,0.017294034322504297,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[39087.38,753.9300000000003,0.019667679272280483,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[38989.83,656.3800000000047,0.017122904408551923,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[38585.19,251.74000000000524,0.0065671104479248605,2,2,4]],[[2024,3,6,16,null,null,null,[-14400]],[38661.05,327.6000000000058,0.008546060946771183,2,2,4]],[[2024,3,7,16,null,null,null,[-14400]],[38791.35,457.90000000000146,0.01194518103640558,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[38722.69,389.24000000000524,0.01015405605287302,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[38769.66,436.2100000000064,0.011379356671523342,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[39005.49,672.0400000000009,0.01753142490435901,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[39043.32,709.8700000000026,0.018518291466069522,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[38905.66,572.2100000000064,0.014927171960781157,2,2,3]],[[2024,3,15,16,null,null,null,[-14400]],[38714.77,381.3199999999997,0.009947447986027862,2,2,4]],[[2024,3,18,16,null,null,null,[-14400]],[38790.43,456.9800000000032,0.011921181109448882,2,2,3]],[[2024,3,19,16,null,null,null,[-14400]],[39110.76,777.310000000005,0.020277590459507427,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[39512.13,1178.6800000000003,0.030748080331929436,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[39781.37,1447.9200000000055,0.03777171112957497,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[39475.9,1142.4500000000044,0.029802952773622108,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[39313.64,980.1900000000023,0.02557009609101196,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[39282.33,948.8800000000047,0.024753315968168915,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[39760.08,1426.6300000000047,0.03721632151554334,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[39807.37,1473.9200000000055,0.03844996993487426,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[39566.85,1233.4000000000015,0.0321755542483132,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[39170.24,836.7900000000009,0.021829237911015077,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[39127.14,793.6900000000023,0.02070489350684591,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[38596.98,263.5300000000061,0.006874674729250984,2,2,4]],[[2024,4,5,16,null,null,null,[-14400]],[38904.04,570.5900000000038,0.014884911219835519,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[38892.8,559.3500000000058,0.014591694720929263,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[38883.67,550.2200000000012,0.014353521532760583,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[38461.51,128.06000000000495,0.003340685484870393,2,2,4]],[[2024,4,11,16,null,null,null,[-14400]],[38459.08,125.63000000000466,0.0032772943734520285,2,2,4]],[[2024,4,12,16,null,null,null,[-14400]],[37983.24,-350.2099999999991,-0.009135885238610121,2,2,4]],[[2024,4,15,16,null,null,null,[-14400]],[37735.11,-598.3399999999965,-0.015608822060106683,2,2,3]],[[2024,4,16,16,null,null,null,[-14400]],[37798.97,-534.4799999999959,-0.01394291408678311,2,2,3]],[[2024,4,17,16,null,null,null,[-14400]],[37753.31,-580.1399999999994,-0.015134040896397258,2,2,3]],[[2024,4,18,16,null,null,null,[-14400]],[37775.38,-558.0699999999997,-0.014558303518206677,2,2,3]],[[2024,4,19,16,null,null,null,[-14400]],[37986.4,-347.04999999999563,-0.00905345070688904,2,2,4]],[[2024,4,22,16,null,null,null,[-14400]],[38239.98,-93.46999999999389,-0.0024383404050507818,2,2,4]],[[2024,4,23,16,null,null,null,[-14400]],[38503.69,170.24000000000524,0.004441029962082861,2,2,4]],[[2024,4,24,16,null,null,null,[-14400]],[38460.92,127.47000000000116,0.0033252942273654254,2,2,4]],[[2024,4,25,16,null,null,null,[-14400]],[38085.8,-247.64999999999418,-0.006460415120475569,2,2,4]],[[2024,4,26,16,null,null,null,[-14400]],[38239.66,-93.7899999999936,-0.002446688205731381,2,2,4]],[[2024,4,29,16,null,null,null,[-14400]],[38386.09,52.63999999999942,0.0013732132119597747,2,2,4]],[[2024,4,30,16,null,null,null,[-14400]],[37815.92,-517.5299999999988,-0.013500741519482303,2,2,3]],[[2024,5,1,16,null,null,null,[-14400]],[37903.29,-430.1599999999962,-0.011221531064905357,2,2,3]],[[2024,5,2,16,null,null,null,[-14400]],[38225.66,-107.7899999999936,-0.002811904485507921,2,2,4]],[[2024,5,3,16,null,null,null,[-14400]],[38675.68,342.2300000000032,0.0089277119591376,2,2,4]],[[2024,5,6,16,null,null,null,[-14400]],[38852.27,518.8199999999997,0.01353439359097602,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[38884.26,550.810000000005,0.01436891279026555,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[39056.39,722.9400000000023,0.01885924695011804,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[39387.76,1054.310000000005,0.027503655423657538,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[39512.84,1179.3899999999994,0.030766602014689507,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[39431.51,1098.060000000005,0.028644956297959226,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[39558.11,1224.6600000000035,0.031947554942224184,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[39908,1574.550000000003,0.04107509238015371,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[39869.38,1535.9300000000003,0.040067617185512924,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[40003.59,1670.1399999999994,0.04356873696471358,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[39806.77,1473.3199999999997,0.03843431780859797,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[39872.99,1539.5400000000009,0.04016179081194103,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[39671.04,1337.5900000000038,0.03489354597616452,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[39065.26,731.810000000005,0.01909063755023367,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[39069.59,736.1399999999994,0.019203593728192987,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[38852.86,519.4100000000035,0.013549784848480987,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[38441.54,108.09000000000378,0.002819730548646255,2,2,4]],[[2024,5,30,16,null,null,null,[-14400]],[38111.48,-221.9699999999939,-0.00579050411585688,2,2,4]],[[2024,5,31,16,null,null,null,[-14400]],[38686.32,352.8700000000026,0.009205276331767756,2,2,4]],[[2024,6,3,16,null,null,null,[-14400]],[38571.03,237.58000000000175,0.006197720267807926,2,2,4]],[[2024,6,4,16,null,null,null,[-14400]],[38711.29,377.8400000000038,0.00985666565362637,2,2,4]],[[2024,6,5,16,null,null,null,[-14400]],[38807.33,473.88000000000466,0.012362049332893458,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[38886.17,552.7200000000012,0.014418738725577824,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[38798.99,465.5400000000009,0.012144484777655049,2,2,3]],[[2024,6,10,16,null,null,null,[-14400]],[38868.04,534.5900000000038,0.013945783643267273,2,2,3]],[[2024,6,11,16,null,null,null,[-14400]],[38747.42,413.97000000000116,0.010799184524221045,2,2,3]],[[2024,6,12,16,null,null,null,[-14400]],[38712.21,378.76000000000204,0.00988066558058307,2,2,4]],[[2024,6,13,16,null,null,null,[-14400]],[38647.1,313.65000000000146,0.008182149010850875,2,2,4]],[[2024,6,14,16,null,null,null,[-14400]],[38589.16,255.7100000000064,0.006670675350118668,2,2,4]],[[2024,6,17,16,null,null,null,[-14400]],[38778.1,444.65000000000146,0.011599529914474212,2,2,3]],[[2024,6,18,16,null,null,null,[-14400]],[38834.86,501.4100000000035,0.013080221060196866,2,2,3]],[[2024,6,20,16,null,null,null,[-14400]],[39134.76,801.310000000005,0.020903675510552926,2,2,3]],[[2024,6,21,16,null,null,null,[-14400]],[39150.33,816.8800000000047,0.021309848187418684,2,2,3]],[[2024,6,24,16,null,null,null,[-14400]],[39411.21,1077.760000000002,0.028115392692283164,2,2,3]],[[2024,6,25,16,null,null,null,[-14400]],[39112.16,778.7100000000064,0.02031411208748512,2,2,3]],[[2024,6,26,16,null,null,null,[-14400]],[39127.8,794.3500000000058,0.020722110845749753,2,2,3]],[[2024,6,27,16,null,null,null,[-14400]],[39164.06,830.6100000000006,0.021668021010370856,2,2,3]],[[2024,6,28,16,null,null,null,[-14400]],[39118.86,785.4100000000035,0.020488894164235244,2,2,3]],[[2024,7,1,16,null,null,null,[-14400]],[39169.52,836.0699999999997,0.021810455359483682,2,2,3]],[[2024,7,2,16,null,null,null,[-14400]],[39331.85,998.4000000000015,0.026045138123492708,2,2,3]],[[2024,7,3,13,null,null,null,[-14400]],[39308,974.5500000000029,0.025422966104016283,2,2,3]],[[2024,7,5,16,null,null,null,[-14400]],[39375.87,1042.4200000000055,0.027193482454618764,2,2,3]],[[2024,7,8,16,null,null,null,[-14400]],[39344.79,1011.3400000000038,0.0263827023135148,2,2,3]],[[2024,7,9,16,null,null,null,[-14400]],[39291.97,958.5200000000041,0.025004793463672174,2,2,3]],[[2024,7,10,16,null,null,null,[-14400]],[39721.36,1387.9100000000035,0.036206237633189906,2,2,3]],[[2024,7,11,16,null,null,null,[-14400]],[39753.75,1420.300000000003,0.03705119158333004,2,2,3]],[[2024,7,12,16,null,null,null,[-14400]],[40000.9,1667.4500000000044,0.04349856326524235,2,2,3]],[[2024,7,15,16,null,null,null,[-14400]],[40211.72,1878.270000000004,0.0489981987011345,2,2,3]],[[2024,7,16,16,null,null,null,[-14400]],[40954.48,2621.030000000006,0.06837448755590761,2,2,3]],[[2024,7,17,16,null,null,null,[-14400]],[41198.08,2864.6300000000047,0.07472925082401936,2,2,3]],[[2024,7,18,16,null,null,null,[-14400]],[40665.02,2331.5699999999997,0.0608233801027562,2,2,3]],[[2024,7,19,16,null,null,null,[-14400]],[40287.53,1954.0800000000017,0.0509758448561244,2,2,3]],[[2024,7,22,16,null,null,null,[-14400]],[40415.44,2081.9900000000052,0.05431261730942572,2,2,3]],[[2024,7,23,16,null,null,null,[-14400]],[40358.09,2024.6399999999994,0.0528165349061981,2,2,3]],[[2024,7,24,16,null,null,null,[-14400]],[39853.87,1520.4200000000055,0.03966300972127491,2,2,3]],[[2024,7,25,16,null,null,null,[-14400]],[39935.07,1601.6200000000026,0.041781264143978765,2,2,3]],[[2024,7,26,16,null,null,null,[-14400]],[40589.34,2255.8899999999994,0.058849125241792734,2,2,3]],[[2024,7,26,17,9,23,null,[-14400]],[40589.34,2255.8899999999994,0.058849125241792734,2,2,3]]]]],null,-14400,39935.07,\"ダウ平均株価\",86400,1]]]",null,null,null,"generic"]]
54
[["di",68],["af.httprm",67,"8191596399748633260",4]]
27
[["e",4,null,null,12421]]
'''

In [6]:
def remove_before_bracket_and_after_newline(s):
    # '['前の文字を削除
    bracket_index = s.find('[')
    if bracket_index != -1:
        s = s[bracket_index:]

    # 改行以降の文字を削除
    newline_index = s.find('\n')
    if newline_index != -1:
        s = s[:newline_index]

    return s

def parse_array_string(array_string):
    def parse_helper(s):
        # 空白文字を除去
        s = s.strip()
        if s.startswith('[') and s.endswith(']'):
            s = s[1:-1]
        else:
            return s.strip()  # Base case: return the stripped string itself

        result = []
        nested_level = 0
        start_index = 0

        for i, char in enumerate(s):
            if char == '[':
                if nested_level == 0:
                    start_index = i
                nested_level += 1
            elif char == ']':
                nested_level -= 1
                if nested_level == 0:
                    result.append(parse_helper(s[start_index:i+1]))
            elif char == ',' and nested_level == 0:
                result.append(parse_helper(s[start_index:i]))
                start_index = i + 1

        if start_index < len(s):
            result.append(parse_helper(s[start_index:]))

        return result

    return parse_helper(array_string)

def index_values(index):
    res_text = remove_before_bracket_and_after_newline(index)

    parsed_array = parse_array_string(res_text)

    return parsed_array[0][2][0][0][4][0][2]

nasdaq = index_values(NASDAQ)
sp500 = index_values(SP500)
vix = index_values(VIX)
dji = index_values(DJI)

# print(res_text)
# print(type(res_text))
# print(type(parsed_array))
# print(len(parsed_array[0][2][0][0][4][0][2]))
# print(parsed_array[0][2][0][0][4][0][2][0])
# print(parsed_array[0][2][0][0][4][0][2][0][0])
# print(parsed_array[0][2][0][0][4][0][2][0][0][0])
# print(parsed_array[0][2][0][0][4][0][2][0][0][1])
# print(parsed_array[0][2][0][0][4][0][2][0][0][2])
# print(parsed_array[0][2][0][0][4][0][2][0][2][0])
print(nasdaq)
print(sp500)
print(vix)
print(dji)

[[['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['17596.27', '0', '0', '2', '2', '2'], ['17596.27', '0', '0', '2', '2', '2']], [['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['17596.27', '0', '0', '2', '2', '2'], ['17596.27', '0', '0', '2', '2', '2']], [['2024', '1', '30', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['2024', '1', '30', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['17476.71', '-119.56000000000131', '-0.006794621814736948', '2', '2', '4'], ['17476.71', '-119.56000000000131', '-0.006794621814736948', '2', '2', '4']], [['2024', '1', '30', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['2024', '1', '30', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['17476.71', '-119.56000000000131', '-0.006794621814736948', '2', '2', '4'], ['174

In [6]:
import json
import requests

BASE_URL = "http://192.168.1.222:8999/"

def test_get_days(stock_code):
    url = f"{BASE_URL}/getdays/{stock_code}"
    response = requests.get(url)
    print(f"GET /getdays/{stock_code} Response: {response.status_code} {response.text}")
    days = response.json()
    dates = list(set(entry['date'] for entry in days))
    print(len(days))
    print(len(dates))
    return dates

def index_for(index, name):
    pre_date = ''
    # JSONデータをforループで回す
    for sp in reversed(index):

        date = f'{sp[0][0]}-{sp[0][1]}-{sp[0][2]}'
        update_value = str(sp[2][0])

        if date != pre_date:
            print(f"Date: {date}, Close: {update_value}")
            data = {
                "date": date,
                "column_name": name,
                "update_value": update_value,
            }

            print("Data to be sent in POST request:", data)

            response = requests.post(f"{BASE_URL}/update_column", json=data)
            print(f"POST /update_column Response: {response.status_code} {response.text}")

            pre_date = date
            if date == '2024-4-15':
                break

index_for(nasdaq, 'NASDAQ')
index_for(sp500, 'SP_500')
index_for(vix, 'vix')
index_for(dji, 'NY_Dow')

Date: 2024-7-26, Close: 19023.656
Data to be sent in POST request: {'date': '2024-7-26', 'column_name': 'NASDAQ', 'update_value': '19023.656'}
POST /update_column Response: 404 No record found for the given date and stock_code
Date: 2024-7-25, Close: 18830.59
Data to be sent in POST request: {'date': '2024-7-25', 'column_name': 'NASDAQ', 'update_value': '18830.59'}
POST /update_column Response: 404 No record found for the given date and stock_code
Date: 2024-7-24, Close: 19032.39
Data to be sent in POST request: {'date': '2024-7-24', 'column_name': 'NASDAQ', 'update_value': '19032.39'}
POST /update_column Response: 404 No record found for the given date and stock_code
Date: 2024-7-23, Close: 19754.34
Data to be sent in POST request: {'date': '2024-7-23', 'column_name': 'NASDAQ', 'update_value': '19754.34'}
POST /update_column Response: 404 No record found for the given date and stock_code
Date: 2024-7-22, Close: 19822.87
Data to be sent in POST request: {'date': '2024-7-22', 'column_na

In [7]:
import json
import requests
from datetime import datetime, timedelta

BASE_URL = "http://192.168.1.222:8999/"

def test_get_days(stock_code):
    url = f"{BASE_URL}/getdays/{stock_code}"
    response = requests.get(url)
    print(f"GET /getdays/{stock_code} Response: {response.status_code} {response.text}")
    days = response.json()
    dates = list(set(entry['date'] for entry in days))
    print(len(days))
    print(len(dates))
    return dates

def index_for(index, name):
    pre_date = ''
    pre_value = ''
    cutoff_date = datetime(2024, 4, 10)  # この日付以前のデータは無視する

    # JSONデータをforループで回す
    for sp in index:
        date = f'{sp[0][0]}-{sp[0][1]}-{sp[0][2]}'
        update_value = str(sp[2][0])

        # 日付の変換
        current_date = datetime.strptime(date, "%Y-%m-%d")

        # 2024-4-14以前のデータは無視
        if current_date <= cutoff_date:
            print(f"Skipping date {date} as it is before or on the cutoff date {cutoff_date.strftime('%Y-%m-%d')}")
            continue

        if pre_date:
            previous_date = datetime.strptime(pre_date, "%Y-%m-%d")
            date_diff = current_date - previous_date

            # 日付が2日以上空いている場合
            if date_diff.days > 1:
                # 空いている日数分、前回の値をPOST
                for i in range(1, date_diff.days):
                    missing_date = previous_date + timedelta(days=i)
                    missing_date_str = missing_date.strftime("%Y-%m-%d")
                    data = {
                        "date": missing_date_str,
                        "column_name": name,
                        "update_value": pre_value.replace(',', ''),  # 前の値を使用
                    }
                    print(f"Posting for missing date: {missing_date_str}, Value: {pre_value}")
                    response = requests.post(f"{BASE_URL}/update_column", json=data)
                    print(f"POST /update_column Response: {response.status_code} {response.text}")

        # 通常のPOST処理
        print(f"Date: {date}, Close: {update_value}")
        data = {
            "date": date,
            "column_name": name,
            "update_value": update_value.replace(',', ''),
        }

        print("Data to be sent in POST request:", data)
        response = requests.post(f"{BASE_URL}/update_column", json=data)
        print(f"POST /update_column Response: {response.status_code} {response.text}")

        # 前回の日付と値を更新
        pre_date = date
        pre_value = update_value

# 使用例
index_for(nasdaq, 'NASDAQ')
index_for(sp500, 'SP_500')
index_for(vix, 'vix')
index_for(dji, 'NY_Dow')


Skipping date 2024-1-29 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-1-29 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-1-30 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-1-30 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-1-31 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-1-31 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-1 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-1 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-2 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-2 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-5 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-5 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-6 as it is before or on the cutoff date 2024-04-10
Skipping date 2024-2-6 as it is before or on the cutoff da

In [31]:
for vix in vix:
    print(vix[0][0], vix[0][1], vix[0][2], vix[2][0])

2024 1 29 13.6
2024 1 29 13.6
2024 1 30 13.31
2024 1 30 13.31
2024 1 31 14.35
2024 1 31 14.35
2024 2 1 13.88
2024 2 1 13.88
2024 2 2 13.85
2024 2 2 13.85
2024 2 5 13.67
2024 2 5 13.67
2024 2 6 13.06
2024 2 6 13.06
2024 2 7 12.83
2024 2 7 12.83
2024 2 8 12.79
2024 2 8 12.79
2024 2 9 12.93
2024 2 9 12.93
2024 2 12 13.93
2024 2 12 13.93
2024 2 13 15.85
2024 2 13 15.85
2024 2 14 14.38
2024 2 14 14.38
2024 2 15 14.01
2024 2 15 14.01
2024 2 16 14.24
2024 2 16 14.24
2024 2 20 15.42
2024 2 20 15.42
2024 2 21 15.34
2024 2 21 15.34
2024 2 22 14.54
2024 2 22 14.54
2024 2 23 13.75
2024 2 23 13.75
2024 2 26 13.74
2024 2 26 13.74
2024 2 27 13.43
2024 2 27 13.43
2024 2 28 13.84
2024 2 28 13.84
2024 2 29 13.4
2024 2 29 13.4
2024 3 1 13.11
2024 3 1 13.11
2024 3 4 13.49
2024 3 4 13.49
2024 3 5 14.46
2024 3 5 14.46
2024 3 6 14.5
2024 3 6 14.5
2024 3 7 14.44
2024 3 7 14.44
2024 3 8 14.74
2024 3 8 14.74
2024 3 11 15.22
2024 3 11 15.22
2024 3 12 13.84
2024 3 12 13.84
2024 3 13 13.75
2024 3 13 13.75
2024 3 1

In [29]:
for nasdaq in nasdaq:
    print(nasdaq[0][0], nasdaq[0][1], nasdaq[0][2], nasdaq[2][0])

2024 1 29 17596.27
2024 1 29 17596.27
2024 1 30 17476.71
2024 1 30 17476.71
2024 1 31 17137.24
2024 1 31 17137.24
2024 2 1 17344.71
2024 2 1 17344.71
2024 2 2 17642.73
2024 2 2 17642.73
2024 2 5 17613.04
2024 2 5 17613.04
2024 2 6 17572.73
2024 2 6 17572.73
2024 2 7 17755.07
2024 2 7 17755.07
2024 2 8 17783.17
2024 2 8 17783.17
2024 2 9 17962.41
2024 2 9 17962.41
2024 2 12 17882.66
2024 2 12 17882.66
2024 2 13 17600.42
2024 2 13 17600.42
2024 2 14 17807.63
2024 2 14 17807.63
2024 2 15 17845.72
2024 2 15 17845.72
2024 2 16 17685.98
2024 2 16 17685.98
2024 2 20 17546.1
2024 2 20 17546.1
2024 2 21 17478.91
2024 2 21 17478.91
2024 2 22 18004.7
2024 2 22 18004.7
2024 2 23 17937.61
2024 2 23 17937.61
2024 2 26 17933.33
2024 2 26 17933.33
2024 2 27 17971.05
2024 2 27 17971.05
2024 2 28 17874.5
2024 2 28 17874.5
2024 2 29 18043.85
2024 2 29 18043.85
2024 3 1 18302.91
2024 3 1 18302.91
2024 3 4 18226.48
2024 3 4 18226.48
2024 3 5 17897.87
2024 3 5 17897.87
2024 3 6 18017.58
2024 3 6 18017.58
20

In [30]:
for sp500 in sp500:
    print(sp500[0][0],sp500[0][1],sp500[0][2],sp500[2][0])

2024 1 29 4927.93
2024 1 29 4927.93
2024 1 30 4924.97
2024 1 30 4924.97
2024 1 31 4845.65
2024 1 31 4845.65
2024 2 1 4906.19
2024 2 1 4906.19
2024 2 2 4958.61
2024 2 2 4958.61
2024 2 5 4942.81
2024 2 5 4942.81
2024 2 6 4954.23
2024 2 6 4954.23
2024 2 7 4995.06
2024 2 7 4995.06
2024 2 8 4997.91
2024 2 8 4997.91
2024 2 9 5026.61
2024 2 9 5026.61
2024 2 12 5021.84
2024 2 12 5021.84
2024 2 13 4953.17
2024 2 13 4953.17
2024 2 14 5000.62
2024 2 14 5000.62
2024 2 15 5029.73
2024 2 15 5029.73
2024 2 16 5005.57
2024 2 16 5005.57
2024 2 20 4975.51
2024 2 20 4975.51
2024 2 21 4981.8
2024 2 21 4981.8
2024 2 22 5087.03
2024 2 22 5087.03
2024 2 23 5088.8
2024 2 23 5088.8
2024 2 26 5069.53
2024 2 26 5069.53
2024 2 27 5078.18
2024 2 27 5078.18
2024 2 28 5069.76
2024 2 28 5069.76
2024 2 29 5096.27
2024 2 29 5096.27
2024 3 1 5137.08
2024 3 1 5137.08
2024 3 4 5130.95
2024 3 4 5130.95
2024 3 5 5078.65
2024 3 5 5078.65
2024 3 6 5104.76
2024 3 6 5104.76
2024 3 7 5157.36
2024 3 7 5157.36
2024 3 8 5123.69
202

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import json
from datetime import datetime

import requests

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

url = "https://www.google.com/finance/?hl=ja"
driver.get(url)

time.sleep(1)

logs = driver.get_log('performance')

stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys("NASDAQ")
stock_code_input.send_keys(Keys.RETURN)
time.sleep(1)


for category in data:
    for company in data[category]:
        stock_code = company["symbol"]
        print(stock_code)

        stock_code_cross = driver.find_element(By.XPATH, r'//*[@id="gb"]/div[2]/div[2]/div/form/button[2]').click()

        stock_code_reflesh = driver.find_element(By.XPATH, r'//*[@id="gb"]/div[2]/div[2]/div/form/div/div/span/div/div/div/div[1]/input[2]')
        # stock_code_reflesh.clear()
        time.sleep(1)

        stock_code_reflesh.send_keys(stock_code)
        stock_code_reflesh.send_keys(Keys.RETURN)
        time.sleep(2)

        current_url = str(driver.current_url)
        print(current_url)
        new_url = f"{current_url}&window=6M"
        driver.get(new_url)
        logs = driver.get_log('performance')
        time.sleep(2)

        batch_request_url = None
        headers = {}
        post_data = None

        for entry in logs:
            log = json.loads(entry['message'])['message']
            try:
                if 'Network.requestWillBeSent' in log['method']:
                    request_url = log['params']['request']['url']
                    if 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=' in request_url and stock_code in request_url:
                        batch_request_url = request_url
                        headers = log['params']['request']['headers']
                        if 'postData' in log['params']['request']:
                            post_data = log['params']['request']['postData']
                        break
            except KeyError:
                continue

        if batch_request_url:
            print(batch_request_url)
            if post_data:
                response = requests.post(batch_request_url, headers=headers, data=post_data)
            else:
                response = requests.get(batch_request_url, headers=headers)

            print("Status Code:", response.status_code)
            print("Response Text:", response.text)
            # try:
            #     data = response.json()
            #     print(data)
            # except json.JSONDecodeError:
            #     print("レスポンスのJSONデコードに失敗しました")
        else:
            print("バッチリクエストURLが見つかりませんでした")


        def remove_before_bracket_and_after_newline(s):
            # '['前の文字を削除
            bracket_index = s.find('[')
            if bracket_index != -1:
                s = s[bracket_index:]

            # 改行以降の文字を削除
            newline_index = s.find('\n')
            if newline_index != -1:
                s = s[:newline_index]

            return s


        def parse_array_string(array_string):
            def parse_helper(s):
                # 空白文字を除去
                s = s.strip()
                if s.startswith('[') and s.endswith(']'):
                    s = s[1:-1]
                else:
                    return s.strip()  # Base case: return the stripped string itself

                result = []
                nested_level = 0
                start_index = 0

                for i, char in enumerate(s):
                    if char == '[':
                        if nested_level == 0:
                            start_index = i
                        nested_level += 1
                    elif char == ']':
                        nested_level -= 1
                        if nested_level == 0:
                            result.append(parse_helper(s[start_index:i+1]))
                    elif char == ',' and nested_level == 0:
                        result.append(parse_helper(s[start_index:i]))
                        start_index = i + 1

                if start_index < len(s):
                    result.append(parse_helper(s[start_index:]))

                return result

            return parse_helper(array_string)


        res_text = remove_before_bracket_and_after_newline(response.text)

        parsed_array = parse_array_string(res_text)

        for data_array in parsed_array[0][2][0][0][4][0][2]:
            print(data_array[0][0], data_array[0][1], data_array[0][2], data_array[2][0])

A
https://www.google.com/finance/quote/A:NYSE?hl=ja
https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=HqGpWd%2Cxh8wxf%2Co6pODe%2CnBEQBc%2CmKsvE%2CuwlMvd%2CSICF5d%2CAiCwsd%2CPr8h2e%2CyYvDpf&source-path=%2Ffinance%2Fquote%2FNDX%3AINDEXNASDAQ&f.sid=-843396336099856088&bl=boq_finance-ui_20240610.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=149257&rt=c
Status Code: 200
Response Text: )]}'

48
[["wrb.fr","yYvDpf","[]",null,null,null,"29"]]
595
[["wrb.fr","HqGpWd","[[[\"/m/02fhwv\",\"ナスダック100指数\",\"ナスダック100指数は、ナスダックに上場する、金融銘柄を除く、時価総額上位100銘柄の時価総額加重平均によって算出される株価指数である。アメリカ合衆国を代表する株価指数の一つとなっている。\",null,null,null,null,null,19032.389,19041.152,19227.2,18721.705,20690.967,14058.333,1393614291,null,null,null,null,null,null,null,null,null,null,0,0,0,1,null,\"https://ja.wikipedia.org/wiki/%E3%83%8A%E3%82%B9%E3%83%80%E3%83%83%E3%82%AF100%E6%8C%87%E6%95%B0\",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,2,2,2,2,0,0,null,null,null,null,n

IndexError: list index out of range

In [4]:
import requests

BASE_URL = "http://192.168.1.222:8999/"

def test_get_days(stock_code):
    url = f"{BASE_URL}/getdays/{stock_code}"
    response = requests.get(url)
    print(f"GET /getdays/{stock_code}:", response.status_code, response.json())
    return response.json()

days = test_get_days("NVDA")
dates = [entry['date'] for entry in days]
dates

ConnectionError: HTTPConnectionPool(host='192.168.1.222', port=8999): Max retries exceeded with url: //getdays/NVDA (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024C553E6A50>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))

In [2]:
import json
import requests

# ファイルを読み込みます
with open('stocks_values.json', 'r') as file:
    data = json.load(file)


BASE_URL = "http://192.168.1.222:8999/"

def test_get_days(stock_code):
    url = f"{BASE_URL}/getdays/{stock_code}"
    response = requests.get(url)
    print(f"GET /getdays/{stock_code}:", response.status_code, response.json())
    return response.json()

# dates = [entry['date'] for entry in days]


# JSONデータをforループで回す
for stock_symbol, entries in data.items():
    print(f"Stock Symbol: {stock_symbol}")

    days = test_get_days(stock_code)
    datas = [entry['date'] for entry in days]

    for entry in entries:
        date = entry['date']
        update_value = entry['close']
        print(f"Date: {date}, Close: {update_value}")

        if date in datas:
            data = {
                "date" : date,
                "column_name" : "value",
                "update_value" : update_value,
                "stock_code" : stock_symbol
            }

            response = requests.post(f"{BASE_URL}/update_column", json=data)
            print(response.status_code, response.text)

Stock Symbol: A
Stock Symbol: AAL
Stock Symbol: AAPL
Stock Symbol: ABBV
Stock Symbol: ABNB
Stock Symbol: ABT
Stock Symbol: ACGL
Stock Symbol: ACN
Stock Symbol: ADBE
Stock Symbol: ADI
Stock Symbol: ADM
Stock Symbol: ADP
Stock Symbol: ADSK
Stock Symbol: AEE
Stock Symbol: AEP
Stock Symbol: AES
Stock Symbol: AFL
Stock Symbol: AIG
Stock Symbol: AIZ
Stock Symbol: AJG
Stock Symbol: AKAM
Stock Symbol: ALB
Stock Symbol: ALGN
Stock Symbol: ALL
Stock Symbol: ALLE
Stock Symbol: AMAT
Stock Symbol: AMCR
Stock Symbol: AMD
Stock Symbol: AME
Stock Symbol: AMGN
Stock Symbol: AMP
Stock Symbol: AMT
Stock Symbol: AMZN
Stock Symbol: ANET
Stock Symbol: ANSS
Stock Symbol: AON
Stock Symbol: AOS
Stock Symbol: APA
Stock Symbol: APD
Stock Symbol: APH
Stock Symbol: APTV
Stock Symbol: ATO
Stock Symbol: AVGO
Stock Symbol: AVY
Stock Symbol: AWK
Stock Symbol: AXON
Stock Symbol: AXP
Stock Symbol: AZO
Stock Symbol: BA
Stock Symbol: BAC
Stock Symbol: BALL
Stock Symbol: BAX
Stock Symbol: BBWI
Stock Symbol: BBY
Stock Symbo